<a href="https://colab.research.google.com/github/mushhub/my-first-blockchain/blob/main/PnL_eth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
import time
from datetime import datetime
import matplotlib.pyplot as plt
from google.colab import forms

# Etherscan APIキーの入力
api_key_input = forms.TextInput(label="Etherscan API Key")
api_key = api_key_input.value

# ウォレットアドレスの入力
wallet_address_input = forms.TextInput(label="ETHウォレットアドレス")
wallet_address = wallet_address_input.value.lower()

def fetch_token_transactions(wallet_address, api_key):
    """ウォレットのERC20トークン取引履歴を取得"""
    url = f"https://api.etherscan.io/api?module=account&action=tokentx&address={wallet_address}&sort=asc&apikey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data["status"] == "1":
            return data["result"]
        else:
            print(f"エラー: {data['message']}")
            return []
    else:
        print(f"APIリクエストエラー: {response.status_code}")
        return []

def fetch_eth_transactions(wallet_address, api_key):
    """ウォレットのETH取引履歴を取得"""
    url = f"https://api.etherscan.io/api?module=account&action=txlist&address={wallet_address}&sort=asc&apikey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data["status"] == "1":
            return data["result"]
        else:
            print(f"エラー: {data['message']}")
            return []
    else:
        print(f"APIリクエストエラー: {response.status_code}")
        return []

def fetch_token_price(token_address, api_key):
    """現在のトークン価格を取得（USD）"""
    url = f"https://api.etherscan.io/api?module=token&action=tokeninfo&contractaddress={token_address}&apikey={api_key}"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            if data["status"] == "1":
                token_info = data["result"][0]
                if "tokenPriceUSD" in token_info:
                    return float(token_info["tokenPriceUSD"])
        # バックアップとしてCoinGeckoを使用
        cg_url = f"https://api.coingecko.com/api/v3/simple/token_price/ethereum?contract_addresses={token_address}&vs_currencies=usd"
        cg_response = requests.get(cg_url)
        if cg_response.status_code == 200:
            cg_data = cg_response.json()
            if token_address.lower() in cg_data:
                return float(cg_data[token_address.lower()]["usd"])
    except Exception as e:
        print(f"価格取得エラー: {e}")
    return None

def fetch_eth_price(api_key):
    """現在のETH価格を取得（USD）"""
    url = f"https://api.etherscan.io/api?module=stats&action=ethprice&apikey={api_key}"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            if data["status"] == "1":
                return float(data["result"]["ethusd"])
    except:
        pass

    # バックアップとしてCoinGeckoを使用
    try:
        cg_url = "https://api.coingecko.com/api/v3/simple/price?ids=ethereum&vs_currencies=usd"
        cg_response = requests.get(cg_url)
        if cg_response.status_code == 200:
            cg_data = cg_response.json()
            return float(cg_data["ethereum"]["usd"])
    except:
        pass

    return None

def analyze_token_transactions(transactions, wallet_address):
    """トークン取引を分析し、PnLを計算"""
    token_balances = {}  # {token_address: {"balance": 0, "symbol": "", "name": "", "decimals": 18, "transactions": []}}

    for tx in transactions:
        token_address = tx["contractAddress"].lower()
        from_address = tx["from"].lower()
        to_address = tx["to"].lower()

        # トークン情報の初期化
        if token_address not in token_balances:
            token_balances[token_address] = {
                "balance": 0,
                "symbol": tx["tokenSymbol"],
                "name": tx["tokenName"],
                "decimals": int(tx["tokenDecimal"]),
                "transactions": [],
                "buy_value_usd": 0,
                "sell_value_usd": 0,
                "transfers_out": 0,
                "transfers_in": 0,
                "gas_spent_eth": 0
            }

        # 取引金額の計算
        value = int(tx["value"]) / (10 ** token_balances[token_address]["decimals"])
        timestamp = int(tx["timeStamp"])
        date = datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

        gas_price = int(tx["gasPrice"])
        gas_used = int(tx["gasUsed"])
        gas_fee_eth = (gas_price * gas_used) / 1e18

        # 取引タイプの判定
        if from_address == wallet_address and to_address == wallet_address:
            tx_type = "自己送金"
            token_balances[token_address]["gas_spent_eth"] += gas_fee_eth
        elif from_address == wallet_address:
            tx_type = "送金/売却"
            token_balances[token_address]["balance"] -= value
            token_balances[token_address]["transfers_out"] += value
            token_balances[token_address]["gas_spent_eth"] += gas_fee_eth
        elif to_address == wallet_address:
            tx_type = "受取/購入"
            token_balances[token_address]["balance"] += value
            token_balances[token_address]["transfers_in"] += value

        # 取引情報の保存
        token_balances[token_address]["transactions"].append({
            "date": date,
            "type": tx_type,
            "value": value,
            "hash": tx["hash"],
            "gas_fee_eth": gas_fee_eth
        })

    return token_balances

def analyze_eth_transactions(transactions, wallet_address):
    """ETH取引を分析"""
    eth_balance = 0
    gas_spent = 0
    eth_transactions = []

    for tx in transactions:
        from_address = tx["from"].lower()
        to_address = tx["to"].lower() if tx["to"] else ""
        value = int(tx["value"]) / 1e18
        gas_price = int(tx["gasPrice"])
        gas_used = int(tx["gasUsed"])
        gas_fee = (gas_price * gas_used) / 1e18
        timestamp = int(tx["timeStamp"])
        date = datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

        if from_address == wallet_address:
            eth_balance -= value
            gas_spent += gas_fee
            tx_type = "送金"
        elif to_address == wallet_address:
            eth_balance += value
            tx_type = "受取"
        else:
            tx_type = "コントラクト実行"
            gas_spent += gas_fee

        eth_transactions.append({
            "date": date,
            "type": tx_type,
            "value": value,
            "hash": tx["hash"],
            "gas_fee": gas_fee
        })

    return {
        "balance": eth_balance,
        "gas_spent": gas_spent,
        "transactions": eth_transactions
    }

def calculate_token_pnl(token_balances, eth_analysis, eth_price):
    """トークンごとのPnLを計算"""
    results = []

    # ETHの分析結果を追加
    results.append({
        "Token": "ETH",
        "Symbol": "ETH",
        "保有数": round(eth_analysis["balance"], 6),
        "送金手数料(ETH)": round(eth_analysis["gas_spent"], 6),
        "送金手数料(USD)": round(eth_analysis["gas_spent"] * eth_price, 2) if eth_price else "N/A",
        "現在価格(USD)": eth_price if eth_price else "N/A",
        "保有価値(USD)": round(eth_analysis["balance"] * eth_price, 2) if eth_price else "N/A",
    })

    # ERC20トークンの分析結果を追加
    for address, data in token_balances.items():
        token_price = fetch_token_price(address, api_key)
        current_value_usd = data["balance"] * token_price if token_price else "N/A"
        gas_fee_usd = data["gas_spent_eth"] * eth_price if eth_price else "N/A"

        # 売却していないトークンは現在の保有数を表示
        if data["balance"] > 0:
            pnl_status = f"保有中: {round(data['balance'], 6)} {data['symbol']}"
        else:
            pnl_status = f"全て売却/送金済み"

        results.append({
            "Token": data["name"],
            "Symbol": data["symbol"],
            "保有数": round(data["balance"], 6),
            "入金合計": round(data["transfers_in"], 6),
            "出金合計": round(data["transfers_out"], 6),
            "送金手数料(ETH)": round(data["gas_spent_eth"], 6),
            "送金手数料(USD)": round(gas_fee_usd, 2) if isinstance(gas_fee_usd, float) else gas_fee_usd,
            "現在価格(USD)": token_price if token_price else "N/A",
            "保有価値(USD)": round(current_value_usd, 2) if isinstance(current_value_usd, float) else current_value_usd,
            "状態": pnl_status
        })

    return results

def visualize_portfolio(results):
    """ポートフォリオを可視化"""
    # 保有価値のあるトークンのみフィルタリング
    portfolio_data = [r for r in results if isinstance(r.get("保有価値(USD)"), float) and r["保有数"] > 0]

    if not portfolio_data:
        print("可視化可能なデータがありません")
        return

    # ポートフォリオ分布の円グラフ
    labels = [f"{r['Symbol']} (${r['保有価値(USD)']})" for r in portfolio_data]
    sizes = [r["保有価値(USD)"] for r in portfolio_data]

    plt.figure(figsize=(10, 6))
    plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
    plt.axis('equal')
    plt.title('ポートフォリオ分布（USD価値ベース）')
    plt.tight_layout()
    plt.show()

def main():
    if not api_key or not wallet_address:
        print("API Keyとウォレットアドレスを入力してください")
        return

    print(f"ウォレットアドレス {wallet_address} の分析を開始します...")

    # ETH取引の取得と分析
    eth_transactions = fetch_eth_transactions(wallet_address, api_key)
    eth_analysis = analyze_eth_transactions(eth_transactions, wallet_address)
    eth_price = fetch_eth_price(api_key)

    # トークン取引の取得と分析
    token_transactions = fetch_token_transactions(wallet_address, api_key)
    token_balances = analyze_token_transactions(token_transactions, wallet_address)

    # PnLの計算
    results = calculate_token_pnl(token_balances, eth_analysis, eth_price)

    # 結果の表示
    print(f"\n結果:")
    df = pd.DataFrame(results)
    print(df)

    # 結果の可視化
    visualize_portfolio(results)

    # 取引履歴の詳細表示（オプション）
    show_details = input("取引履歴の詳細を表示しますか？ (y/n): ").lower() == 'y'
    if show_details:
        print("\nETH取引履歴:")
        eth_tx_df = pd.DataFrame(eth_analysis["transactions"])
        print(eth_tx_df)

        for address, data in token_balances.items():
            print(f"\n{data['name']} ({data['symbol']}) 取引履歴:")
            token_tx_df = pd.DataFrame(data["transactions"])
            print(token_tx_df)

if __name__ == "__main__":
    main()

ImportError: cannot import name 'forms' from 'google.colab' (/usr/local/lib/python3.11/dist-packages/google/colab/__init__.py)